In [4]:
import os
import requests
from bs4 import BeautifulSoup

# URL to be scraped
urls = ['https://edition.cnn.com/2023/03/24/tech/china-opposes-tiktok-sale-approval-needed-intl-hnk', 'https://edition.cnn.com/2023/03/21/tech/tiktok-national-security-concerns', 'https://edition.cnn.com/2023/03/17/tech/bytedance-criminal-investigation', 'https://edition.cnn.com/2023/03/24/tech/twitter-verified-checkmarks', 'https://edition.cnn.com/2023/03/24/tech/tiktok-douyin-bytedance-china-intl-hnk', 'https://edition.cnn.com/2023/03/24/tech/tiktok-ban-national-security-hearing', 'https://edition.cnn.com/2023/03/23/tech/south-korea-do-kwon-terra-fraud-arrest-hnk-intl', 'https://edition.cnn.com/2022/09/07/tech/apple-september-event-highlights-trends', 'https://edition.cnn.com/2022/09/08/business/chevrolet-equinox-ev', 'https://edition.cnn.com/tech/live-news/apple-iphone-14-event', 'https://edition.cnn.com/2022/09/07/business/tesla-euro-ncap-autopilot', 'https://edition.cnn.com/2023/03/24/cars/eu-combustion-engine-debate-climate-intl', 'https://edition.cnn.com/2023/03/24/tech/china-opposes-tiktok-sale-approval-needed-intl-hnk']

for index, url in enumerate(urls):
    # Send a GET request to the URL and retrieve the response
    response = requests.get(url)

    # Check if the response is successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the article title
        title = soup.find('h1').text

        # Extract the article text
        article_text = ''
        for paragraph in soup.find_all('p'):
            article_text += paragraph.text + '\n'

        # Save the extracted text to a file with URL_ID as its file name
        file_path = r'F:\DATA\Interview Assignment\Text Analysis\Text File'
        file_name = url.split('/')[-1]+ '.txt'
        file_full_path = os.path.join(file_path, file_name)
        with open(file_full_path, 'w', encoding='utf-8') as f:
            print(file_name)
            #print(title + '\n\n' + article_text)
            f.write(title + '\n\n' + article_text)
        f.close()

    else:
        print("Failed to retrieve the URL.",url," Response code:",response.status_code)



china-opposes-tiktok-sale-approval-needed-intl-hnk.txt
tiktok-national-security-concerns.txt
bytedance-criminal-investigation.txt
twitter-verified-checkmarks.txt
tiktok-douyin-bytedance-china-intl-hnk.txt
tiktok-ban-national-security-hearing.txt
south-korea-do-kwon-terra-fraud-arrest-hnk-intl.txt
apple-september-event-highlights-trends.txt
chevrolet-equinox-ev.txt
apple-iphone-14-event.txt
tesla-euro-ncap-autopilot.txt
eu-combustion-engine-debate-climate-intl.txt
china-opposes-tiktok-sale-approval-needed-intl-hnk.txt


In [11]:
import string
import nltk
#nltk.download("punkt")
from nltk.tokenize import word_tokenize
import os
import pandas as pd

# Function to load words from a file and return as a list
def load_words(file_path):
    with open(file_path, "r") as file:
        words = list(file.read().splitlines())
    return words

# Function to calculate sentiment of the text
def calculate_sentiment(clean_text, stop_words, positive_words, negative_words):
    positive_count = 0
    negative_count = 0
    for word in clean_text:
        if word in stop_words:
            continue
        elif word in positive_words:
            positive_count += 1
        elif word in negative_words:
            negative_count += 1
    if positive_count > negative_count:
        return "Positive"
    elif positive_count < negative_count:
        return "Negative"
    else:
        return "Neutral"

# Function to perform sentimental analysis on the text
def sentimental_analysis(words, positive_words, negative_words):

        positive_score = 0
        negative_score = 0
        total_words = len(words)
        # calculate positive and negative scores
        for word in words:
            if word in positive_words:
                positive_score += 1
            elif word in negative_words:
                negative_score += 1

        # calculate polarity score
        polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)

        # calculate subjectivity score
        subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)

        return {'positive_score' : positive_score,'negative_score' : negative_score, 
                'polarity_score' : polarity_score, 'subjectivity_score' : subjectivity_score} 

stop_words = load_words("stop_words.txt")
positive_words = load_words("positive-words.txt")
negative_words = load_words("negative-words.txt")




# Function to perform readability analysis on the text
def readability_analysis(text_words,article):
    words = text_words 
    num_words = len(words)
    #print(num_words)
    num_sentences = article.count(".") + article.count("!") + article.count("?")
    #print(num_sentences)
    avg_sentence_length = num_words / num_sentences
    
    # Function to perform syllable in word
    def syllables_in_word(word):
        word = word.lower()
        vowels = "aeiou"
        count = 0
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count += 1
        if word.endswith("es") or word.endswith("ed"):
            count -= 1
        if word.endswith("e"):
            count -= 1
        if count == 0:
            count += 1
        return count

    # Count the number of complex words in the text
    complex_word_count = 0
    for word in words:
        if syllables_in_word(word) > 2:
            complex_word_count += 1
            
    # Calculate the percentage of complex words       
    percentage_complex_words = complex_word_count / num_words
    
    # Calculate the average number of syllables per word
    syllable_count_per_word = sum([syllables_in_word(word) for word in words]) / len(words)
    
    # Count the number of personal pronouns in the text
    personal_pronouns = 0
    personal_pronouns_list = ['I', 'we', 'my', 'ours', 'us']
    for word in words:
        if word.lower() in personal_pronouns_list:
            personal_pronouns += 1
    
    # Calculate the average word length
    total_chars = 0
    for word in words:
        total_chars += len(word)
    avg_word_len = total_chars / len(words)
    
    # Function to calculate average number of words per sentence
    tokens = tokenize_text
    sentences = nltk.sent_tokenize(Article_text)
    num_words = len(tokens)
    num_sentences = len(sentences)
    avg_words_per_sentence = num_words / num_sentences
    
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    
    return {'avg_sentence_length' : avg_sentence_length,
            'percentage_complex_words' : percentage_complex_words,
            'fog_index' : fog_index,
            'avg_words_per_sentence' : avg_words_per_sentence,
            'complex_word_count' : complex_word_count,
            'word_count' : len(tokenize_text),
            'syllable_count_per_word' : syllable_count_per_word,
            'personal_pronouns' : personal_pronouns,
            'avg_word_len' : avg_word_len}
            
# Directory containing the .txt files
directory = r'F:\DATA\Interview Assignment\Text Analysis\Text File\text_file'

# Create a list to store the analysis data
analysis_data = []

# Loop through all the .txt files in the directory
for file_name in os.listdir(directory):
    if file_name.endswith('.txt'):
        file_path = os.path.join(directory, file_name)
        print(file_name)
        # Open the .txt file and perform the analysis
        with open(file_path, 'r', encoding='UTF-8') as f:
            Article_text = f.read()
            
            # Import text from text file
            #Article_text = open("man-and-machines-together-machines-are-more-diligent-than-humans.txt", encoding="utf-8").read()
            ###print(article_text)
            
            #clean the text file
            ###clean_text = text.translate(str.maketrans("", "", string.punctuation)).lower()

            #Tokenize text using word_tokenizer
            tokenize_text = word_tokenize(Article_text)

            # Remove punctuation and convert to lowercase
            clean_text = [word.lower() for word in tokenize_text if word not in string.punctuation]

            # Calculate sentiment of the text
            sentiment = calculate_sentiment(tokenize_text, stop_words, positive_words, negative_words)
            print("Sentiment:", sentiment)
            
            # Perform the sentiment analysis
            sentiment_analysis_out = sentimental_analysis(tokenize_text, positive_words, negative_words)
            print(sentiment_analysis_out)
            
            # Perform the readability analysis
            readability_analysis_out = readability_analysis(tokenize_text, Article_text)
            
            # Combine the analysis data and append to the list
            analysis = {'file_name': file_name,**sentiment_analysis_out, **readability_analysis_out}
            analysis_data.append(analysis)

# Convert the analysis data list to a pandas DataFrame
df = pd.DataFrame(analysis_data)
print(df)
# Save the DataFrame to an Excel file
file_path = r'F:\DATA\Interview Assignment\Text Analysis\Output_Data.xlsx'
df.to_excel(file_path, sheet_name="sheet3", index=False)





apple-iphone-14-event.txt
Sentiment: Positive
{'positive_score': 45, 'negative_score': 17, 'polarity_score': 0.4516128959417275, 'subjectivity_score': 0.03615160347746262}
apple-september-event-highlights-trends.txt
Sentiment: Neutral
{'positive_score': 27, 'negative_score': 20, 'polarity_score': 0.14893616704391135, 'subjectivity_score': 0.030779305808265024}
bytedance-criminal-investigation.txt
Sentiment: Negative
{'positive_score': 3, 'negative_score': 10, 'polarity_score': -0.5384614970414233, 'subjectivity_score': 0.021885521848677574}
chevrolet-equinox-ev.txt
Sentiment: Positive
{'positive_score': 13, 'negative_score': 4, 'polarity_score': 0.5294117335640156, 'subjectivity_score': 0.01963048496578466}
china-opposes-tiktok-sale-approval-needed-intl-hnk.txt
Sentiment: Positive
{'positive_score': 15, 'negative_score': 12, 'polarity_score': 0.11111110699588492, 'subjectivity_score': 0.03529411760092272}
eu-combustion-engine-debate-climate-intl.txt
Sentiment: Negative
{'positive_score